# Run the SONAR-netCDF4 v1 convention checker on an AZFP file

In [1]:
import echopype as ep
from echopype.testing import TEST_DATA_FOLDER

from sonarnetcdf4_echopype_checker import cc

In [2]:
import datetime

print(f"echopype version: {ep.__version__}")
print(f"{datetime.datetime.utcnow()} +00:00")

echopype version: 0.7.2.dev65+ga10d22bf
2023-08-27 21:55:56.594750 +00:00


## Convert the target raw file to an EchoData object

In [3]:
# Rutgers glider data, with empty tilt_x and tilt_y
# ed = ep.open_raw(
#     TEST_DATA_FOLDER / 'azfp/rutgers_glider_notemperature/18011107.01A', 
#     xml_path=TEST_DATA_FOLDER / 'azfp/rutgers_glider_notemperature/18011107.XML', 
#     sonar_model='AZFP'
# )

# A more normal data, with valid tilt_x and tilt_y
ed = ep.open_raw(
    TEST_DATA_FOLDER / 'azfp/17082117.01A', 
    xml_path=TEST_DATA_FOLDER / 'azfp/17041823.XML', 
    sonar_model='AZFP'
)

In [4]:
ed

<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains backscatter power (uncalibrated) and other beam or channel-specific data.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

## Global obligation settings

In [5]:
obligation_target = "M"  # "M", ['M', 'MA'], None

## Test the `Sonar/Beam_group1` group

### Read and process the CDL

In [6]:
target_group = "Sonar/Beam_group1"

In [7]:
conv_check = cc.ConventionCDL(target_group)

### Examine the target group in the converted EchoData object

In [8]:
conv_check.set_ed_group_ds(ed[target_group])

In [9]:
conv_check.ed_group_ds

<xarray.Dataset>
Dimensions:                      (channel: 4, ping_time: 240, range_sample: 2650)
Coordinates:
  * channel                      (channel) <U11 '55075-38-1' ... '55075-455-4'
  * ping_time                    (ping_time) datetime64[ns] 2017-08-21T17:05:...
  * range_sample                 (range_sample) int64 0 1 2 3 ... 2647 2648 2649
Data variables: (12/16)
    frequency_nominal            (channel) float64 3.8e+04 1.25e+05 ... 4.55e+05
    beam_type                    (channel) int64 0 0 0 0
    beam_direction_x             (channel) float64 nan nan nan nan
    beam_direction_y             (channel) float64 nan nan nan nan
    beam_direction_z             (channel) float64 nan nan nan nan
    backscatter_r                (channel, ping_time, range_sample) float32 2...
    ...                           ...
    transmit_frequency_start     (channel) float64 3.8e+04 1.25e+05 ... 4.55e+05
    transmit_frequency_stop      (channel) float64 3.8e+04 1.25e+05 ... 4.55e+05
    transmit_type                <U2 'CW'
    beam_stabilisation           int8 0
    non_quantitative_processing  int16 0
    sample_time_offset           float64 0.0
Attributes:
    beam_mode:              
    conversion_equation_t:  type_4

### Set the obligation and echopype_mods filters (optional)

In [10]:
conv_check.set_obligation(obligation_target) # "M", ['M', 'MA'], None, exclude=True

conv_check._get_obligation_vars()

array(['backscatter_r', 'beam', 'beam_direction_x', 'beam_direction_y',
       'beam_direction_z', 'beam_stabilisation', 'beam_type',
       'equivalent_beam_angle', 'non_quantitative_processing',
       'ping_time', 'sample_interval', 'sample_time_offset',
       'transmit_duration_nominal', 'transmit_frequency_start',
       'transmit_frequency_stop', 'transmit_type'], dtype=object)

`echopype_mods` handling is not fully implemented yet

In [11]:
conv_check.set_echopype_mods(None) # None

In [12]:
conv_check.echopype_mods

### Run the tests

#### Presence of expected variables

In [13]:
conv_check.test_vars_presence(test_type="expected")

****Expected variables not found in the EchoData object:
beam


#### Presence of unexpected variables

In [14]:
conv_check.test_vars_presence(test_type="unexpected")

****EchoData variables not found in the CDL:


#### Variable data type

Use two types of data type comparisons: strict (specific) vs generalized.

In [15]:
conv_check.test_vars_datatype(dtype_strict=True)

****Variables with different data type from what is expected:
transmit_duration_nominal: EchoData type: float64, CDL type: float32
transmit_frequency_start: EchoData type: float64, CDL type: float32
transmit_type: EchoData type: <U2, CDL type: object
transmit_frequency_stop: EchoData type: float64, CDL type: float32
sample_time_offset: EchoData type: float64, CDL type: float32


In [16]:
conv_check.test_vars_datatype(dtype_strict=False)

****Variables with different data type from what is expected:
transmit_type: EchoData type: <U2, CDL type: object


#### Variable dimensionality

In [17]:
conv_check.test_vars_dimensionality()

****EchoData variables with dimensionality different from the CDL:
transmit_frequency_start: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
beam_type: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time')
transmit_type: EchoData dims: (), CDL dims: ('channel', 'ping_time')
beam_direction_x: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
beam_direction_y: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
equivalent_beam_angle: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
transmit_frequency_stop: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
beam_direction_z: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
backscatter_r: EchoData dims: ('channel', 'ping_time', 'range_sample'), CDL dims: ('channel', 'ping_time', 'range_sample', 'beam')
beam_stabilisation: EchoData dims: (), CDL dims: ('channel', 'ping_time')
sample_time_offset: Echo

#### Attribute presence

In [18]:
conv_check.test_attrs_presence()

****Variable or global missing convention attributes:
beam_type: Missing EchoData attrs: set()
transmit_type: Missing EchoData attrs: set()
beam_stabilisation: Missing EchoData attrs: set()


In [19]:
conv_check.test_attrs_presence(global_attrs=True)

****Variable or global missing convention attributes:


#### Attribute values

In [20]:
conv_check.test_attrs_value()

****Variable or global attribute with different values:
beam_type.long_name: EchoData value: Beam type
backscatter_r.units: EchoData value: count
non_quantitative_processing.flag_meanings: EchoData value: ['None']


In [21]:
conv_check.test_attrs_value(global_attrs=True)

****Variable or global attribute with different values:
global.beam_mode: EchoData value: 
global.conversion_equation_t: EchoData value: type_4


## Test the `Platform` group

### Read and process the CDL

In [22]:
target_group = "Platform"

In [23]:
conv_check = cc.ConventionCDL(target_group)

### Examine the target group in the converted EchoData object

In [24]:
conv_check.set_ed_group_ds(ed[target_group])

In [25]:
conv_check.ed_group_ds

<xarray.Dataset>
Dimensions:              (time1: 1, time2: 240, channel: 4)
Coordinates:
  * channel              (channel) <U11 '55075-38-1' ... '55075-455-4'
  * time1                (time1) datetime64[ns] 2017-08-21T17:05:37
  * time2                (time2) datetime64[ns] 2017-08-21T17:05:37 ... 2017-...
Data variables: (12/21)
    latitude             (time1) float64 nan
    longitude            (time1) float64 nan
    pitch                (time2) float64 nan nan nan nan nan ... nan nan nan nan
    roll                 (time2) float64 nan nan nan nan nan ... nan nan nan nan
    vertical_offset      (time2) float64 nan nan nan nan nan ... nan nan nan nan
    water_level          float64 nan
    ...                   ...
    MRU_rotation_y       float64 nan
    MRU_rotation_z       float64 nan
    position_offset_x    float64 nan
    position_offset_y    float64 nan
    position_offset_z    float64 nan
    frequency_nominal    (channel) float64 3.8e+04 1.25e+05 2e+05 4.55e+05
Attributes:
    platform_name:       
    platform_type:       
    platform_code_ICES:

In [26]:
ed

<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains backscatter power (uncalibrated) and other beam or channel-specific data.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

### Set the obligation and echopype_mods filters (optional)

In [27]:
conv_check.set_obligation(obligation_target) # "M", ['M', 'MA'], None, exclude=True

conv_check._get_obligation_vars()

array([], dtype=object)

`echopype_mods` handling is not fully implemented yet

In [28]:
conv_check.set_echopype_mods(None) # None

In [29]:
conv_check.echopype_mods

### Run the tests

#### Presence of expected variables

In [30]:
conv_check.test_vars_presence(test_type="expected")

****Expected variables not found in the EchoData object:


#### Presence of unexpected variables

In [31]:
conv_check.test_vars_presence(test_type="unexpected")

****EchoData variables not found in the CDL:


#### Variable data type

Use two types of data type comparisons: strict (specific) vs generalized.

In [32]:
conv_check.test_vars_datatype(dtype_strict=True)

****Variables with different data type from what is expected:


In [33]:
conv_check.test_vars_datatype(dtype_strict=False)

****Variables with different data type from what is expected:


#### Variable dimensionality

In [34]:
conv_check.test_vars_dimensionality()

****EchoData variables with dimensionality different from the CDL:


#### Attribute presence

In [35]:
conv_check.test_attrs_presence()

****Variable or global missing convention attributes:


In [36]:
conv_check.test_attrs_presence(global_attrs=True)

****Variable or global missing convention attributes:


#### Attribute values

In [37]:
conv_check.test_attrs_value()

****Variable or global attribute with different values:


In [38]:
conv_check.test_attrs_value(global_attrs=True)

****Variable or global attribute with different values:
